In [2]:
from tabulate import tabulate
from reservoirpy.nodes import (Reservoir, IPReservoir, NVAR, LMS, RLS, Ridge, ESN)
from reservoirpy.datasets import (lorenz, mackey_glass, narma)
from reservoirpy.observables import (rmse, rsquare, nrmse, mse)
import numpy as np
import copy
import math
from numpy import linalg as LA
import pandas as pd
from bayes_opt import BayesianOptimization

In [39]:
def getData():
    sunspots = pd.read_csv("../datasets/Sunspots.csv")
    data = sunspots.loc[:,"Monthly Mean Total Sunspot Number"]
    data1 = np.expand_dims(data[:-20], axis=1)
    data2 = np.expand_dims(data[10:-10], axis=1)
    data3 = np.expand_dims(data[20:], axis=1)
    y = data3[1:]
    x = np.concatenate([data1, data2, data3], axis=-1)[:-1]

    trainX = x[:2100]
    trainY = y[:2100]
    testX = x[2100:]
    testY = y[2100:]
    return trainX, trainY, testX, testY

In [40]:
def testDefault():
    def blackbox(**params):
        esn = Reservoir(units=int(params['units']), sr=params['sr'], rc_connectivity=params['rc_connectivity']) >> Ridge(output_dim=1, ridge=params['ridge'])
        x, y, _, _ = getData()
        trainX = x[:1100]
        trainY = y[:1100]
        valX = x[1100:]
        valY = y[1100:]

        esn.fit(trainX, trainY, warmup=100)
        preds = esn.run(valX)
        return 1/nrmse(valY, preds)
    
    pbounds = {'units': (100, 1000), 'sr': (0.1, 1.2), 'rc_connectivity': (0.008, 0.02), 'ridge': (1e-6, 1e-4)}
    
    optimizer = BayesianOptimization(
        f=blackbox,
        pbounds=pbounds,
        random_state=1,
    )

    optimizer.maximize(
        init_points=20,
        n_iter=25,
    )

    params = optimizer.max['params']
    bestEsn = Reservoir(units=int(params['units']), sr=params['sr'], rc_connectivity=params['rc_connectivity']) >> Ridge(output_dim=1, ridge=params['ridge'])
    trainX, trainY, testX, testY = getData()
    bestEsn.fit(trainX, trainY, warmup=100)
    preds = bestEsn.run(testX)
    return nrmse(testY, preds)


In [58]:
def testFeedback():
    def blackbox(**params):
        reservoir = Reservoir(units=int(params['units']), sr=params['sr'], rc_connectivity=params['rc_connectivity'])
        readout = Ridge(output_dim=1, ridge=params['ridge'])
        # reservoir <<= readout
        esn = reservoir >> readout

        x, y, _, _ = getData()
        trainX = x[:1100]
        trainY = y[:1100]
        valX = x[1100:]
        valY = y[1100:]

        esn.fit(trainX, trainY, warmup=100)
        preds = esn.run(valX)
        return 1/nrmse(valY, preds)
    
    pbounds = {'units': (100, 1000), 'sr': (0.1, 1.2), 'rc_connectivity': (0.008, 0.02), 'ridge': (1e-6, 1e-4)}
    
    optimizer = BayesianOptimization(
        f=blackbox,
        pbounds=pbounds,
        random_state=1,
    )

    optimizer.maximize(
        init_points=20,
        n_iter=25,
    )

    params = optimizer.max['params']
    reservoir = Reservoir(units=int(params['units']), sr=params['sr'], rc_connectivity=params['rc_connectivity'])
    readout = Ridge(output_dim=1, ridge=params['ridge'])
    # reservoir <<= readout
    bestEsn = reservoir >> readout
    trainX, trainY, testX, testY = getData()
    bestEsn.fit(trainX, trainY, warmup=100)
    preds = bestEsn.run(testX)
    return nrmse(testY, preds)

print(testFeedback())

|   iter    |  target   | rc_con... |   ridge   |    sr     |   units   |
-------------------------------------------------------------------------


Running Model-807: 1100it [00:00, 20626.68it/s]         
Running Model-807: 100%|██████████| 1/1 [00:00<00:00, 17.25it/s]


Fitting node Ridge-612...


Running Model-807: 999it [00:00, 21745.79it/s]          


| 1         | 5.253     | 0.013     | 7.231e-05 | 0.1001    | 372.1     |


...

| 45        | 3.008     | 0.009574  | 3.593e-05 | 0.883     | 503.5     |


Running Model-852: 2099it [00:00, 22227.50it/s]         
Running Model-852: 100%|██████████| 1/1 [00:00<00:00, 10.11it/s]


Fitting node Ridge-657...


Running Model-852: 1164it [00:00, 23005.88it/s]         

0.19638893806765492


In [53]:
def testProposed():
    def blackbox(**params):
        esn = (NVAR(delay=int(params['delay']), order=int(params['order']), strides=int(params['strides'])) >> 
               Reservoir(units=int(params['units']), sr=params['sr'], rc_connectivity=params['rc_connectivity'])) >> Ridge(output_dim=1, ridge=params['ridge'])
        x, y, _, _ = getData()
        trainX = x[:1100]
        trainY = y[:1100]
        valX = x[1100:]
        valY = y[1100:]

        esn.fit(trainX, trainY, warmup=100)

        esn2 = (NVAR(delay=int(params['delay']), order=int(params['order']), strides=int(params['strides'])) >> 
               Reservoir(units=int(params['units']), sr=params['sr'], rc_connectivity=params['rc_connectivity'])) >> Ridge(output_dim=1, ridge=params['ridge'])
        esnPreds = copy.deepcopy(esn).run(trainX)
        esn2.fit(esnPreds, trainY, warmup=100)

        preds1 = esn.run(valX)
        preds2 = esn2.run(preds1)
        return 1/nrmse(valY, preds2)
    
    pbounds = {'units': (100, 1000), 'sr': (0.1, 1.2), 'rc_connectivity': (0.008, 0.02), 'ridge': (1e-6, 1e-4), 'delay': (2, 12), 'order': (2, 5), 'strides': (1, 6)}
    
    optimizer = BayesianOptimization(
        f=blackbox,
        pbounds=pbounds,
        random_state=1,
    )

    optimizer.maximize(
        init_points=20,
        n_iter=25,
    )

    params = optimizer.max['params']
    bestEsn1 = (NVAR(delay=int(params['delay']), order=int(params['order']), strides=int(params['strides'])) >> 
               Reservoir(units=int(params['units']), sr=params['sr'], rc_connectivity=params['rc_connectivity'])) >> Ridge(output_dim=1, ridge=params['ridge'])
    bestEsn2 = (NVAR(delay=int(params['delay']), order=int(params['order']), strides=int(params['strides'])) >> 
               Reservoir(units=int(params['units']), sr=params['sr'], rc_connectivity=params['rc_connectivity'])) >> Ridge(output_dim=1, ridge=params['ridge'])
    trainX, trainY, testX, testY = getData()
    trainX1 = trainX[:1100]
    trainY1 = trainY[:1100]
    teacherForceX = trainX[1100:]
    teacherForceY = trainY[1100:]
    bestEsn1.fit(trainX1, trainY1, warmup=100)
    outputs = bestEsn1.run(teacherForceX, forced_feedbacks=teacherForceY)
    bestEsn2.run(outputs, forced_feedbacks=teacherForceY)
    preds1 = bestEsn1.run(testX)
    preds2 = bestEsn2.run(preds1)
    return nrmse(testY, preds2)

In [42]:
print(testDefault())

|   iter    |  target   | rc_con... |   ridge   |    sr     |   units   |
-------------------------------------------------------------------------


Running Model-289: 1100it [00:00, 21762.90it/s]         
Running Model-289: 100%|██████████| 1/1 [00:00<00:00, 18.33it/s]


Fitting node Ridge-284...


Running Model-289: 999it [00:00, 21566.82it/s]          


| 1         | 5.231     | 0.013     | 7.231e-05 | 0.1001    | 372.1     |


...

| 45        | 5.298     | 0.01039   | 7.096e-06 | 0.125     | 480.7     |


Running Model-334: 2099it [00:00, 17530.83it/s]?, ?it/s]
Running Model-334: 100%|██████████| 1/1 [00:00<00:00,  7.45it/s]


Fitting node Ridge-329...


Running Model-334: 1164it [00:00, 19045.53it/s]         

0.1952812290090029


In [41]:
def testDefault():
    esn = Reservoir(units=1000, sr=0.8, rc_connectivity=0.01, noise_in=1e-10) >> Ridge(output_dim=1, ridge=1e-5)
    trainX, trainY, testX, testY = getData()

    esn.fit(trainX, trainY, warmup=1000)
    preds = esn.run(testX)
    return nrmse(testY, preds)

def testDeep():
    esn = (Reservoir(units=1000, sr=0.8, rc_connectivity=0.01, noise_in=1e-10) >> Reservoir(units=1000, sr=0.8, rc_connectivity=0.01, noise_in=1e-10)) >> Ridge(output_dim=1, ridge=1e-5)
    trainX, trainY, testX, testY = getData()

    esn.fit(trainX, trainY, warmup=1000)
    preds = esn.run(testX)
    return nrmse(testY, preds)

def testDeepCascade():
    esn = (Reservoir(units=1000, sr=0.8, rc_connectivity=0.01, noise_in=1e-10) >> Reservoir(units=1000, sr=0.8, rc_connectivity=0.01, noise_in=1e-10)) >> Ridge(output_dim=1, ridge=1e-5)
    trainX, trainY, testX, testY = getData()

    esn.fit(trainX, trainY, warmup=1000)

    esn2 = (Reservoir(units=1000, sr=0.8, rc_connectivity=0.01, noise_in=1e-10) >> Reservoir(units=1000, sr=0.8, rc_connectivity=0.01, noise_in=1e-10)) >> Ridge(output_dim=1, ridge=1e-5)
    esnPreds = copy.deepcopy(esn).run(trainX)
    esn2.fit(esnPreds, trainY, warmup=1000)

    preds1 = esn.run(testX)
    preds2 = esn2.run(preds1)
    return nrmse(testY, preds2)

def testCascade():
    esn = Reservoir(units=1000, sr=0.8, rc_connectivity=0.01, noise_in=1e-10) >> Ridge(output_dim=1, ridge=1e-5)
    trainX, trainY, testX, testY = getData()

    esn.fit(trainX, trainY, warmup=1000)

    esn2 = Reservoir(units=1000, sr=0.8, rc_connectivity=0.01, noise_in=1e-10) >> Ridge(output_dim=1, ridge=1e-5)
    esnPreds = copy.deepcopy(esn).run(trainX)
    esn2.fit(esnPreds, trainY, warmup=1000)

    preds1 = esn.run(testX)
    preds2 = esn2.run(preds1)
    return nrmse(testY, preds2)


def testDeepCascadeNVAR():
    esn = (NVAR(delay=8, order=3, strides=4) >> Reservoir(units=1000, sr=0.8, rc_connectivity=0.01, noise_in=1e-10)) >> Ridge(output_dim=1, ridge=1e-5)
    trainX, trainY, testX, testY = getData()

    esn.fit(trainX, trainY, warmup=1000)

    esn2 = (NVAR(delay=8, order=3, strides=4) >> Reservoir(units=1000, sr=0.8, rc_connectivity=0.01, noise_in=1e-10)) >> Ridge(output_dim=1, ridge=1e-5)
    esnPreds = copy.deepcopy(esn).run(trainX)
    esn2.fit(esnPreds, trainY, warmup=1000)

    preds1 = esn.run(testX)
    preds2 = esn2.run(preds1)
    return nrmse(testY, preds2)

def testNVAR():
    esn = NVAR(delay=8, order=3, strides=4) >> Ridge(output_dim=1, ridge=1e-5)
    trainX, trainY, testX, testY = getData()

    esn.fit(trainX, trainY, warmup=1000)
    preds = esn.run(testX)
    return nrmse(testY, preds)

error1 = testDefault()
error2 = testCascade()
error4 = testDeep()
error5 = testDeepCascade()
error6 = testDeepCascadeNVAR()
error7 = testNVAR()
print("==================Lorenz NRMSE==================")
print('Normal training error:', error1)
print('NVAR training error', error7)
print('Cascading ESN error:', error2)
print('Deep ESN error:', error4)
print('Deep + cascade error:', error5)
print('Deep + cascade + NVAR error:', error6)

Running Model-2: 2100it [00:00, 15452.40it/s]?, ?it/s]
Running Model-2: 100%|██████████| 1/1 [00:00<00:00,  6.72it/s]


Fitting node Ridge-2...


Running Model-2: 1144it [00:00, 16751.68it/s]         
Running Model-3: 2100it [00:00, 15784.11it/s]?, ?it/s]
Running Model-3: 100%|██████████| 1/1 [00:00<00:00,  6.81it/s]


Fitting node Ridge-3...


Running Model-3: 2100it [00:00, 16559.58it/s]         
Running Model-4: 2100it [00:00, 15872.48it/s]?, ?it/s]
Running Model-4: 100%|██████████| 1/1 [00:00<00:00,  6.84it/s]


...

Fitting node Ridge-9...


Running Model-12: 1144it [00:00, 3349.89it/s]          
Running Model-14: 1144it [00:00, 9856.89it/s]          
Running Model-15: 2100it [00:00, 13629.42it/s]?, ?it/s]
Running Model-15: 100%|██████████| 1/1 [00:00<00:00,  4.76it/s]
c:\Users\Alok\Downloads\phd\ESN_research\venv\lib\site-packages\reservoirpy\nodes\readouts\ridge.py:17: LinAlgWarning: Ill-conditioned matrix (rcond=2.94756e-25): result may not be accurate.
  return linalg.solve(XXT + ridge, YXT.T, assume_a="sym")


Fitting node Ridge-10...


Running Model-15: 1144it [00:00, 14864.85it/s]         

==================Lorenz NRMSE==================
Normal training error: 0.7166276174273154
NVAR training error 23.442228545655794
Cascading ESN error: 1.0129983148546795
Deep ESN error: 4.044797242784084
Deep + cascade error: 11.654705661606103
Deep + cascade + NVAR error: 0.8302470218958352


In [63]:
numEvals = 20
trainSize = 3000
teacherForcing = 1000

def testMGSDeepCascadeNVAR():
    nvar1 = NVAR(delay=8, order=3, strides=4)
    reservoir1 = Reservoir(units=1000, sr=0.8, rc_connectivity=0.01, noise_in=1e-10)
    readout1 = Ridge(output_dim=1, ridge=1e-5, name="readout1-1")
    nvar2 = NVAR(delay=8, order=3, strides=4)
    reservoir2 = Reservoir(units=1000, sr=0.8, rc_connectivity=0.01, noise_in=1e-10)
    readout2 = Ridge(output_dim=1, ridge=1e-5, name="readout2-1")
    esn = nvar1 >> reservoir1 >> readout1 >> nvar2 >> reservoir2 >> readout2
    data = np.array(mackey_glass(n_timesteps=(10 * trainSize), h=0.1))
    data = data[::10]
    data = np.tanh(data-1)
    X = data[:-1]
    Y = data[1:]

    esn.fit(X, {"readout1-1": Y, "readout2-1": Y}, warmup=1000)

    error = 0
    for i in range(numEvals):
        data2 = np.array(mackey_glass(n_timesteps=10 * (teacherForcing+85), seed=i, h=0.1))
        data2 = data2[::10]
        data2 = np.tanh(data2-1)

        teacherForcingX = data2[:teacherForcing]
        teacherForcingY = data2[1:teacherForcing+1]
        x_test = data2[teacherForcing:-1]
        y_test = data2[teacherForcing+84:]
        esn.run(teacherForcingX, forced_feedbacks=teacherForcingY, reset=True)

        pred = esn.run(x_test)[-1]
        error += (y_test[0][0] - pred)**2
    variance = np.var(data)
    return math.sqrt(error/(numEvals*variance))

print(testMGSDeepCascadeNVAR())

Running Model-877: 2999it [00:00, 8174.95it/s]<?, ?it/s]
Running Model-877: 100%|██████████| 1/1 [00:00<00:00,  2.59it/s]


Fitting node readout1-1...


Running Model-877: 2999it [00:00, 7465.84it/s]<?, ?it/s]
Running Model-877: 100%|██████████| 1/1 [00:00<00:00,  2.39it/s]


Fitting node readout2-1...


Running Model-877: 1000it [00:00, 4465.45it/s]          
Running Model-877: 84it [00:00, 4927.23it/s]            
Running Model-877: 1000it [00:00, 4473.58it/s]          
Running Model-877: 84it [00:00, 4930.61it/s]            
Running Model-877: 1000it [00:00, 4386.62it/s]          
Running Model-877: 84it [00:00, 5195.41it/s]            
Running Model-877: 1000it [00:00, 4453.41it/s]          
Running Model-877: 84it [00:00, 4813.73it/s]            
Running Model-877: 1000it [00:00, 4406.52it/s]          
Running Model-877: 84it [00:00, 5237.42it/s]            
Running Model-877: 1000it [00:00, 4390.92it/s]          
Running Model-877: 84it [00:00, 4944.52it/s]            
Running Model-877: 1000it [00:00, 4274.97it/s]          
Running Model-877: 84it [00:00, 4844.77it/s]            
Running Model-877: 1000it [00:00, 4436.23it/s]          
Running Model-877: 84it [00:00, 4732.32it/s]            
Running Model-877: 1000it [00:00, 4384.06it/s]          
Running Model-877: 84it [00:00,

3.806589397601199e-05
